In [3]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream

In [4]:
import numpy as np
import imutils
import time
import cv2
import os

In [13]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    #grab the dimensions of the frame and contruct a blob from it
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224,224), (104.0,177.0,123.0))
    
    #pass the blob through the network and obtain face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    #print(detections.shape)
    
    #list of faces, corresponding location and prediction
    faces = []
    locs = []
    preds = []
    
    #loop over the detections
    for i in range(0, detections.shape[2]):
        #extract the confidence/probability associated with thr detection
        confidence = detections[0,0,i,2]
        
        #filer weak detections
        if confidence > 0.5:
            #compute x,y coordinates for box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')
            
            #ensure box lies within frame boundary
            (startX,startY) = (max(0,startX),max(0,startY))
            (endX,endY) = (min(w-1,endX),min(h-1,endY))
            
            #face ROI, ordering, resizing and preprocess
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            faces.append(face)
            locs.append((startX, startY, endX, endY))
        
    #prediction if atleast one face is detected
    if len(faces) > 0:
        faces = np.array(faces,dtype='float32')
        preds = maskNet.predict(faces, batch_size=32)
            
    return(locs,preds)

In [14]:
#load our serialized face detector model from disk
prototxtPath = r'deploy.prototxt'
weightsPath = r'res10_300x300_ssd_iter_140000.caffemodel'

In [15]:
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [16]:
#load the facemask detector model from the disk
maskNet = load_model('mask_detector.model')

In [17]:
#initialising the video stream
vs = VideoStream(src = 0).start()

#loop over the frames from the videostream
while True:
    #grab the frame from the video and rezise it with max width of 400
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    
    #detect faces and fetermine face mask
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    
    #loop over the detected face and corresponding locations
    for (box,pred) in zip(locs, preds):
        #unpack the boz and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        
        #determine class label and color of the box
        label = 'Mask' if mask > withoutMask else 'No Mask'
        color = (0,255,0) if label == 'Mask' else (0,0,255)
        
        #include the probability in the label
        label = '{} : {:.2f}%'.format(label, max(mask,withoutMask)*100)
        
        #display the label and bounding box on the output frame
        cv2.putText(frame, label, (startX, startY-10),
                   cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(frame,(startX, startY),(endX, endY),color,2)
    
    #show the output frame
    cv2.imshow('Frame',frame)
    key = cv2.waitKey(1) & 0xFF
    
    #if the 'q' key was pressed, break from the loop
    if key == ord('q'):
        break
#cleanup
cv2.destroyAllWindows()
vs.stop()